In [3]:
def epsilon_closure(transition, states):
    epsilon_states = set()
    stack = list(states)

    while stack:
        state = stack.pop()
        if state in epsilon_states:
            continue
        epsilon_states.add(state)
        if 'ε' in transition[state]:
            stack.extend(transition[state]['ε'])

    return epsilon_states

def epsilon_nfa_to_nfa(epsilon_transition, start_state, final_states):
    nfa_transition = {}

    # Initialize the NFA states
    for state in epsilon_transition.keys():
        nfa_transition[state] = {}

    for state in epsilon_transition.keys():
        for symbol in epsilon_transition[state]:
            if symbol != 'ε' and symbol != 'null':
                nfa_transition[state][symbol] = set()

                for eps_state in epsilon_transition[state][symbol]:
                    if eps_state != 'null':
                        epsilon_states = epsilon_closure(epsilon_transition, [eps_state])
                        nfa_transition[state][symbol].update(epsilon_states)

    nfa_start_state = start_state
    nfa_final_states = set()

    for state in final_states:
        epsilon_states = epsilon_closure(epsilon_transition, [state])
        nfa_final_states.update(epsilon_states)

    return nfa_transition, nfa_start_state, nfa_final_states

def print_transition_table(transition):
    states = sorted(transition.keys())
    symbols = set()

    # Collect all symbols used in transitions
    for state in transition:
        for symbol in transition[state]:
            symbols.add(symbol)

    symbols = sorted(list(symbols))

    # Print the table header
    header = ['State'] + symbols
    print()
    print("NFA Transition Table:")
    print("\t".join(header))

    # Print the table rows
    for state in states:
        row = [state]
        for symbol in symbols:
            if symbol in transition[state]:
                next_states = transition[state][symbol]
                next_states_list = list(next_states)
                next_states_str = ", ".join(next_states_list)
                row.append("{{{}}}".format(next_states_str))
            else:
                row.append("-")
        print("\t\t".join(row))

if __name__ == "__main__":
    epsilon_transition = {}

    # Input format: (source state, symbol) -> [destination states or 'null' for no transition]
    while True:
        input_str = input("Enter transition (source, symbol, destination or 'null') or 'done' to finish: ").strip()
        if input_str.lower() == "done":
            break

        source, symbol, dest = input_str.split(",")
        source, dest = source.strip(), dest.strip()

        if source not in epsilon_transition:
            epsilon_transition[source] = {}

        if symbol not in epsilon_transition[source]:
            epsilon_transition[source][symbol] = set()

        if dest.lower() == 'null':
            epsilon_transition[source][symbol].add('null')
        else:
            epsilon_transition[source][symbol].add(dest)

    start_state = input("Enter the start state of the ε-NFA: ").strip()
    final_states = input("Enter final state(s) of the ε-NFA separated by commas: ").strip().split(",")

    nfa_transition, nfa_start_state, nfa_final_states = epsilon_nfa_to_nfa(epsilon_transition, start_state, final_states)

    print_transition_table(nfa_transition)

    print(f"\nStart state in NFA: {nfa_start_state}")
    print(f"Final state(s) in NFA: {', '.join(nfa_final_states)}")



Enter transition (source, symbol, destination or 'null') or 'done' to finish: q0,1,q1
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q0,ε,q2
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q1,1,q0
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q2,0,q3
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q2,1,q4
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q3,0,q2
Enter transition (source, symbol, destination or 'null') or 'done' to finish: q4,0,q2
Enter transition (source, symbol, destination or 'null') or 'done' to finish: done
Enter the start state of the ε-NFA: q0
Enter final state(s) of the ε-NFA separated by commas: q2

NFA Transition Table:
State	0	1
q0		-		{q1}
q1		-		{q2, q0}
q2		{q3}		{q4}
q3		{q2}		-
q4		{q2}		-

Start state in NFA: q0
Final state(s) in NFA: q2
